# PIC-SURE API use-case: quick analysis on CIBMTR data

This is a tutorial notebook aimed to get the user quickly up and running with the R PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE R API
### What is PIC-SURE?

As part of the BioData Catalyst initiative, the Patient Information Commons Standard Unification of Research Elements (PIC-SURE) platform has been integrating clinical and genomic datasets from multiple TOPMed and TOPMed related studies funded by the National Heart Lung and Blood Institute (NHLBI).

Original data exposed through PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By simplifying the process of data extraction, it allows investigators to focus on the downstream analyses and to facilitate reproducible sciences.


### More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the databases the same way using either language.


PIC-SURE is a larger project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface that allows researchers to explore variables across multiple studies, filter patients that match criteria, and create cohorts from this interactive exploration.

The R API is actively developed by the Avillach Lab at Harvard Medical School.

PIC-SURE API GitHub repo:
* https://github.com/hms-dbmi/pic-sure-r-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-r-client
* https://github.com/hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds




 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

# Environment set-up

### Pre-requisites
- R 3.4 or later

### Install packages

Install the following:
- packages listed in the `requirements.R` file
- PIC-SURE API components (from Github)
    - PIC-SURE Adapter
    - PIC-SURE Client

In [ ]:
source("R_lib/requirements.R")

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
install.packages("https://cran.r-project.org/src/contrib/Archive/devtools/devtools_1.13.6.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/R6_2.5.0.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/hash_2.2.6.1.tar.gz", repos=NULL, type="source")
install.packages(c("urltools"),repos = "http://cran.us.r-project.org")
devtools::install_github("hms-dbmi/pic-sure-r-client", force=T)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force=T)
devtools::install_github("hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds", force=T)

##### Loading user-defined functions

In [ ]:
source("R_lib/utils.R")

## Connecting to a PIC-SURE resource

The following is required to get access to data through the PIC-SURE API: 
- Network URL
- Resource id
- User-specific security token

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token) file.

In [ ]:
# Set required information as variables
PICSURE_network_URL <- "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
#PICSURE_network_URL <- "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id <- "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file <- "token.txt"

In [ ]:
token <- scan(token_file, what = "character")

In [ ]:
# Establish connection to PIC-SURE
connection <- picsure::connect(url = PICSURE_network_URL,
                                 token = token)

In [ ]:
# it may take several minutes to connect and download the initialization data
resource <- bdc::get.resource(connection,
                               resourceUUID = resource_id)

Two objects are created here: a `connection` and a `resource` object.

Since we will only be using a single resource, **the `resource` object is actually the only one we will need to proceed with data analysis hereafter**.

It is connected to the specific data source ID we specified and enables us to query and retrieve data from this database.

## Getting help with the R PIC-SURE API

You can get help with PIC-SURE library functions by using the `?` operator.

In [ ]:
?hpds::get.resource()

## Using the *variables dictionary*

Once a connection to the desired resource has been established, we first need to understand which variables are available in the database. To this end, we will use the `find.in.dictionary` function.

For instance, looking for variables containing the term `Sex` is done this way:

In [ ]:
dictionary_search <- bdc::find.in.dictionary(resource, "Sex")

Four different functions can be used to retrieve results from a dictionary search: `extract.count()`, `extract.keys()`, `extract.entries()`, and `extract.dataframe()`.

In [ ]:
print(list("Count"   = bdc::extract.count(dictionary_search), 
           "Keys"    = bdc::extract.keys(dictionary_search)[1:2], # Show first two keys
           "Entries" = bdc::extract.entries(dictionary_search)[1:2])) # Show first two entries

In [ ]:
bdc::extract.entries(dictionary_search) %>% tail() #View last entries as a dataframe

**`bdc::extract.dataframe()` retrieves the result of the dictionary search in a data.frame format. This way, it enables us to:**

* Use the various information exposed in the dictionary (patientCount, variable type ...) as criteria for variable selection.
* Use the row names of the DataFrame to get the actual variables names, to be used in the query, as shown below.

Variable names, as currently implemented in the API, aren't very practical to use right away for two reasons:
1. Very long
2. Presence of backslashes that requires modification right after copy-pasting.

However, using the dictionary to select variables can help access the variable names. Let's say we want to retrieve every variable from the HCT for SCD dataset. One way to proceed is to retrieve the whole dictionary for those variables in the form of a data.frame, as below:

In [ ]:
plain_variablesDict <-  hpds::find.in.dictionary(resource, "HCT for SCD") %>% 
                        hpds::extract.entries()

In [ ]:
plain_variablesDict[1:5,]

The dictionary currently returned by the API provides information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if strings, False if numerical
- min/max: only provided for numerical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently, the API only exposes 'phenotypes' variables

### Extract full data dictionary to CSV

Using the `bdc::find.in.dictionary` we can extact the entire data dictionary by performing an empty search and saving it to `fullVariablesDict`:

In [ ]:
fullVariablesDict <- hpds::find.in.dictionary(resource, "") %>% 
                    hpds::extract.entries()# %>%
                    #mutate(categoryValues = stri_join_list(categoryValues, sep =', '))
dim(fullVariablesDict)

Check that the `fullVariablesDict` dataframe contains some values.

In [ ]:
fullVariablesDict[0:5,] # View first five rows

We can than write the data frame that contains the full data dictionary to a csv file.

In [ ]:
dataDictFile <- "data_dictionary_R.csv" # Name of output file
saveDictFrame <- fullVariablesDict[ , c("name", "patientCount", "min", "categorical", "observationCount", "max", "HpdsDataType", "description")]
write.csv(saveDictFrame, dataDictFile, row.names = FALSE)

You should now see a `data_dictionary_R.csv` in the Jupyter Hub file explorer, in the same folder as this notebook.

### Parsing variable names

We can use a simple function, `get_multiIndex_variablesDict`, defined in `R_lib/utils.R` to add a little more information to the variable dictionary and to simplify working with variables names.

Although not an official feature of the API, such functionality illustrates how to quickly select groups of related variables.

Printing part of the parsed names dictionary allows us to quickly see the tree-like organization of the variable names. Moreover, original and simplified variable names are now stored respectively in the `varName` and `simplified_varName` columns (simplified variable names is simply the last component of the variable name, that is usually the most informative to know what each variable is about).

In [ ]:
# Display the variables tree hierarchy from the variables name
variablesDict <- get_multiIndex_variablesDict(plain_variablesDict)
head(variablesDict)

Below is a simple example to illustrate the ease of use a multiIndex dictionary. Let's say we are interested in the variables within the "5 - CRF data collection track only" of the "Hematopoietic Cell Transplant for Sickle Cell Disease (HCT for SCD) ( phs002385 )" study.

In [ ]:
mask_study <- variablesDict[,1] == "Hematopoietic Cell Transplant for Sickle Cell Disease (HCT for SCD) ( phs002385 )"
mask_dctrack <- grepl("5 - CRF data collection track only", variablesDict[["level_1"]])
more_variables <- variablesDict[mask_study & mask_dctrack,]
more_variables

This simple filter can be easily combined with other filters to quickly select variables of interest.

## Querying and retrieving data

The second cornerstone of the API are the `query` functions, which is how we retrieve data from the resource.

First, we need to create a query object.

In [ ]:
my_query <- bdc::new.query(resource=resource)

The query function has several methods that enable us to build a query:

| Method | Arguments / Input | Output|
|--------|-------------------|-------|
| query.select.add() | variable names (string) or list of strings | all variables included in the list (no record subsetting)|
| query.require.add() | variable names (string) or list of strings | all variables; only records that do not contain null values for input variables |
| query.anyof.add() | variable names (string) or list of strings | all variables; only records that contain at least one non-null value for input variables |
| query.filter.add() | variable name and additional filtering values | input variable; only records that match filter criteria |

All 4 methods can be combined when building a query. The record eventually returned by the query have to meet all the different specified filters.

### Building the query

Let's say we are interested in the age at which patients from the following cohort received their transplant:
* males
* patients with avascular necrosis
* patients that received their transplant after the year 1999

First we will find variables pertaining to sex and avascular necrosis. We can do this by searching for "Sex" and "Avascular necrosis" in the `simplified_name` column of `variablesDict`.

In [ ]:
sex_var <- variablesDict[variablesDict["simplified_name"] == "Sex", ]$name %>% as.character()
avascular_necrosis_varname <- variablesDict[variablesDict["simplified_name"] == "Avascular necrosis", ]$name %>% as.character() 

Next, we can find the variable pertaining to "Year of transplant".

In [ ]:
yr_transplant_varname <- variablesDict[variablesDict["simplified_name"] == "Year of transplant", ]$name %>% as.character()

Now we can create a new query and apply our filters to retrieve the cohort of interest.

In [ ]:
bdc::query.filter.add(query = my_query, 
                      keys = sex_var,
                      value = "Male")

bdc::query.filter.add(query = my_query, 
                      keys = avascular_necrosis_varname,
                      value = "Yes")

bdc::query.filter.add(query = my_query, 
                      keys = yr_transplant_varname, 
                      min=2000)

Using this cohort, we can add the variable of interest: "Patient age at transplant, years"

In [ ]:
age_transplant_var = variablesDict[variablesDict['simplified_name'] == "Patient age at transplant, years",]$name %>% as.character()
bdc::query.select.add(my_query, keys = age_transplant_var)

## Retrieving the data

Once our query object is finally built, we use the `query.run` function to retrieve the data corresponding to our query

In [ ]:
my_df <- hpds::query.run(my_query, result.type = "dataframe")

In [ ]:
my_df

Once the data has been retrieved as a dataframe, you can use R functions to conduct analyses and create visualizations, such as this:

In [ ]:
names(my_df)[2] <- "age_at_transplant" # Rename long column to age_at_transplant
ggplot(data = my_df) +
    geom_histogram(mapping = aes(x=age_at_transplant), bins=15) +
    labs(x = "Age received transplant, yrs old", y = "Count") +
    theme_bw()